In [ ]:
!pip uninstall -y numpy spacy thinc pyresparser ahpy --quiet


In [ ]:
# Clean and install all compatible packages
!pip uninstall -y numpy spacy thinc --quiet
!pip install thinc==7.4.5 --quiet
!pip install pyresparser==1.0.6 nltk==3.6.7 python-docx ahpy --quiet


In [ ]:
!pip uninstall -y numpy spacy thinc


In [ ]:
# Compatible numpy version for TensorFlow, OpenCV, etc.
!pip install numpy==1.26.4

# Compatible spaCy and thinc versions
!pip install spacy==3.7.4 thinc==8.2.2


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import spacy
spacy.cli.download("en_core_web_sm")


In [ ]:
!pip install PyMuPDF --quiet

In [ ]:
# Install OCR dependencies
!pip install pdf2image pytesseract Pillow --quiet
!apt-get install poppler-utils -y


In [ ]:
# --- Imports ---
import fitz  # PyMuPDF
import os
import re
import docx
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
from difflib import SequenceMatcher
from pyresparser import ResumeParser
import string
import random

from pdf2image import convert_from_path
import pytesseract
from PIL import Image

In [ ]:
import numpy as np
import spacy

print("NumPy version:", np.__version__)
print("spaCy loaded:", spacy.blank("en"))


In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Paths ---
zip_path = "/content/drive/MyDrive/Total_Resumes.zip"
extract_to = "/content/resumes_unzipped"
resume_folder = os.path.join(extract_to, "Total_Resumes")

# --- Unzip resumes ---
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# --- Collect only PDF resumes ---
pdf_files = [
    os.path.join(resume_folder, f)
    for f in os.listdir(resume_folder)
    if f.lower().endswith(".pdf")
]

# --- Extract Text Functions ---
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")

    if not text.strip():
        print(f"No text extracted from {pdf_path}, trying OCR...")
        text = extract_text_with_ocr(pdf_path)

    return text

def extract_text_with_ocr(pdf_path):
    try:
        pages = convert_from_path(pdf_path)
        ocr_text = ""
        for page in pages:
            ocr_text += pytesseract.image_to_string(page)
        return ocr_text
    except Exception as e:
        print(f"OCR failed for {pdf_path}: {e}")
        return ""


# --- Strict Role Extraction (as in Shiny) ---
import re
import string
import random

# List of plausible job title keywords
job_keywords = [
    "engineer", "developer", "manager", "analyst", "consultant", "specialist", "coordinator",
    "executive", "associate", "administrator", "advisor", "officer", "lead", "head", "director",
    "scientist", "architect", "designer", "supervisor", "technician", "trainer", "assistant"
]

fallback_titles = [
    "Engineer", "Analyst", "Developer", "Executive", "Consultant", "Manager", "Data Scientist", "Software Engineer"
]
# --- Strict Role Extraction (as in Shiny) ---
invalid_role_keywords = [
    # Degrees/fields
    'b.tech', 'btech', 'b.e', 'be', 'bachelor', 'master', 'msc', 'm.sc', 'mba', 'pgdm', 'phd',
    'engineering', 'computer science', 'information technology', 'electronics', 'civil', 'mechanical',
    # Section headers
    'education', 'skills', 'projects', 'experience', 'certifications', 'achievements', 'summary',
    'objective', 'profile', 'about me', 'contact', 'personal details',
    # Soft skills/qualities
    'leadership', 'leadership qualities', 'leadership quality', 'communication', 'team player',
    'problem solving', 'quick learner', 'hardworking', 'presentation skills', 'good communication',
    # Formatting artifacts
    '•', '', '', '-', '_', '—', '–', '•', '·', '…', '“', '”', '‘', '’',
    # Company names (add your own if needed)
    'leadsquared',
    # Misc
    'simply', 'trainee', 'intern', 'fresher'
]


def is_valid_role(line):
    l = line.lower().strip()
    l = l.strip(string.punctuation + "•-—–·…“”‘’")
    # Exclude lines with academic titles or references
    if any(x in l for x in ['advisor:', 'prof.', 'dr.', 'reference', 'referee', 'guide', 'supervisor']):
        return False
    if not (1 <= len(l.split()) <= 6):
        return False
    if any(char.isdigit() for char in l) or "http" in l or "www" in l:
        return False
    for kw in invalid_role_keywords:
        if kw in l:
            return False
    if l.isupper() or l.split()[0] in ['education', 'skills', 'projects', 'experience', 'summary', 'objective', 'profile']:
        return False
    if any(jk in l for jk in job_keywords):
        return True
    return False

def extract_most_recent_role(text):
    date_pattern = re.compile(
        r'((?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})\s*[-–to]+\s*(present|current|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})',
        re.IGNORECASE
    )
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    for i, line in enumerate(lines):
        if date_pattern.search(line):
            for j in range(1, 3):
                if i - j >= 0:
                    candidate = lines[i - j]
                    for part in re.split(r'\||,| and |/| at ', candidate):
                        part = part.strip()
                        if is_valid_role(part):
                            return part.title()
    for line in lines:
        for part in re.split(r'\||,| and |/| at ', line):
            part = part.strip()
            if is_valid_role(part):
                return part.title()
    return random.choice(fallback_titles)

def clean_role(role):
    if not isinstance(role, str):
        return role
    # Remove only true prefixes
    role = re.sub(r'^(designation:|role:|certificate by:|advisor:)\s*', '', role, flags=re.I)
    # Split on common delimiters and take the first plausible part
    for delim in ['|', ',', '&', ' and ', '/', ' at ']:
        if delim in role:
            parts = [p.strip() for p in role.split(delim)]
            for part in parts:
                if is_valid_role(part):
                    return part.title()
            return parts[0].title()
    role = role.strip().strip('.')
    return role.title()

def guess_role_from_text(text):
    for line in text.splitlines():
        l = line.strip().lower()
        if not (1 <= len(l.split()) <= 3):
            continue
        if any(char.isdigit() for char in l):
            continue
        if any(k in l for k in invalid_role_keywords):
            continue
        if any(jk in l for jk in job_keywords):
            return line.strip().title()
    return random.choice(fallback_titles)



def clean_current_role(raw_role, resume_text):
    # Use parser's role if valid and not concatenated
    if raw_role and isinstance(raw_role, str):
        for part in re.split(r'\||,| and |/| at ', raw_role):
            part = part.strip()
            if is_valid_role(part):
                return clean_role(part)
    # Try context-based extraction
    context_role = extract_most_recent_role(resume_text)
    if context_role:
        return clean_role(context_role)
    # Fallback
    return random.choice(fallback_titles)

# --- Resume Feature Extraction ---
def extract_resume_features_combined(file_path, resume_text):
    lowered = resume_text.lower()
    lines = resume_text.splitlines()
    date_pattern = re.compile(
        r'((?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})\s*[-–to]+\s*(present|current|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})',
        re.IGNORECASE
    )

    total_months = 0
    latest_start = None
    job_count = 0

    for line in lines:
        match = date_pattern.search(line)
        if match:
            try:
                start_str, end_str = match.groups()
                start_date = datetime.strptime(start_str.strip()[:3] + start_str.strip()[-4:], '%b%Y') if '/' not in start_str else datetime.strptime(start_str.strip(), '%m/%Y')
                end_date = datetime.now() if 'present' in end_str.lower() else datetime.strptime(end_str.strip()[:3] + end_str.strip()[-4:], '%b%Y') if '/' not in end_str else datetime.strptime(end_str.strip(), '%m/%Y')
                if start_date <= end_date:
                    delta = relativedelta(end_date, start_date)
                    months = delta.years * 12 + delta.months
                    total_months += months
                    job_count += 1
                    if latest_start is None or start_date > latest_start:
                        latest_start = start_date
            except:
                continue

    current_tenure_months = 0
    if latest_start:
        delta = relativedelta(datetime.now(), latest_start)
        current_tenure_months = delta.years * 12 + delta.months

    total_exp = int(round(total_months / 12))
    current_tenure = int(round(current_tenure_months / 12))

    # --- Fallbacks and Consistency ---
    parser_exp = 0
    current_role = ""
    skills = []

    try:
        parsed = ResumeParser(file_path).get_extracted_data()
        print(f"{os.path.basename(file_path)} parser output: {parsed}")  # Debugging
        # --- Role extraction ---
        raw_role = parsed.get('designation', "")
        if isinstance(raw_role, list):
            raw_role = raw_role[0] if raw_role else ""
        role_from_parser = clean_current_role(raw_role, resume_text)
        # --- Skills extraction ---
        skills = parsed.get('skills', [])
        # --- Experience fallback ---
        parser_exp = parsed.get('total_experience', 0) or 0
    except Exception as e:
        print(f"ResumeParser failed for {os.path.basename(file_path)}: {e}")
        role_from_parser = ""
        skills = []

    # --- Try context-based and fallback role extraction if needed ---
    role_from_context = extract_most_recent_role(resume_text)
    role_from_guess = guess_role_from_text(resume_text)

    # --- Choose the first valid, non-empty role ---
    for candidate in [role_from_parser, role_from_context, role_from_guess]:
        if candidate and is_valid_role(candidate):
            current_role = candidate
            break
    else:
        current_role = random.choice(fallback_titles)

    # --- Experience/tenure fallback logic ---
    if total_exp == 0 and current_tenure == 0 and parser_exp:
        total_exp = int(round(parser_exp))
        current_tenure = total_exp

    if total_exp == 0 and current_tenure > 0:
        total_exp = current_tenure
    if current_tenure == 0 and total_exp > 0:
        current_tenure = total_exp
    if current_tenure > total_exp:
        current_tenure = total_exp

    # Try to extract from phrases like "X years of experience"
    if total_exp == 0 and current_tenure == 0:
        exp_phrase = re.search(r'(\d+)\s+years? of experience', lowered)
        if exp_phrase:
            total_exp = int(exp_phrase.group(1))
            current_tenure = total_exp

    # Final log if still zero
    if total_exp == 0 and current_tenure == 0:
        print(f"Could not extract experience for {os.path.basename(file_path)}")
    certs = len(re.findall(r'\b(certification|certificate|certified)\b', lowered))
    pubs = len(re.findall(r'\b(publication|journal|conference|research)\b', lowered))
    skill_keywords = ['python', 'sql', 'excel', 'communication', 'crm', 'presentation', 'ai', 'cloud']
    skill_density = sum(lowered.count(skill) for skill in skill_keywords)

    return {
        'current_role': current_role,
        'total_experience': total_exp,
        'current_tenure': current_tenure,
        'certification_count': certs,
        'publication_count': pubs,
        'skill_density': skill_density,
        'job_count': job_count,
        'skills': skills
    }

import os
import re
import string

# --- Blacklist to avoid junk names ---
blacklist = {
    "curriculum", "resume", "btech", "b.e", "be", "mba", "mca", "mtech", "cse", "ece", "eee",
    "civil", "mech", "ece", "student", "intern", "fresher", "developer", "engineer", "job",
    "file", "document", "doc", "new", "final", "latest", "updated", "resume1", "resume2"
}

def extract_person_name(resume_text, resume_path, email=None, parsed_name=None):
    def clean_name(name):
        name = name.lower().strip()
        name = re.sub(r'[^a-z ]+', '', name)  # Remove non-letter chars
        name = re.sub(r'\s+', ' ', name)      # Normalize spaces
        name = name.title()
        return name

    # --- Try parsed name if valid ---
    if parsed_name:
        name = clean_name(parsed_name)
        if name.lower() not in blacklist and len(name.split()) <= 4 and len(name.split()) >= 1:
            return name

    # --- Try extracting from email ---
    if email:
        username = email.split("@")[0]
        username = username.replace(".", " ").replace("_", " ").replace("-", " ")
        name = clean_name(username)
        parts = name.split()
        # Remove digits and blacklist
        parts = [p for p in parts if p.lower() not in blacklist and not any(char.isdigit() for char in p)]
        if 1 <= len(parts) <= 3:
            return " ".join([p.title() for p in parts])
    return "Unknown"



In [ ]:
import sqlite3
import json
import os
from tqdm import tqdm

# --- Connect to new DB ---
db_path = "/content/sample_data/Resumeparser.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- Drop and recreate the parsed_resumes table with additional fields ---
cursor.execute("DROP TABLE IF EXISTS parsed_resumes")
conn.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS parsed_resumes (
    person_name TEXT,
    resume_path TEXT,
    current_role TEXT,
    total_experience INTEGER,
    current_tenure INTEGER,
    certification_count INTEGER,
    publication_count INTEGER,
    skill_density INTEGER,
    full_parsed_json TEXT,
    fitment_score INTEGER,
    fitment_level TEXT,
    email TEXT
)
""")
conn.commit()

# --- Parse and store resumes ---
results = []

for file in tqdm(pdf_files, desc="📄 Parsing resumes"):
    text = extract_text_from_pdf(file)
    if not text.strip():
        print(f"No text extracted from {os.path.basename(file)}")
        continue

    features = extract_resume_features_combined(file, text)

    try:
        parsed = ResumeParser(file).get_extracted_data()
        applicant_name = parsed.get("name", "")
        email = parsed.get("email", "")

        if not applicant_name:
            if email and "@" in email:
                applicant_name = email.split("@")[0].replace(".", " ").title()
    except:
        parsed = {}
        email = ""
        applicant_name = os.path.basename(file).split("@")[0].split(".")[0].title()

    # Insert into new DB including the new 'email' field
    cursor.execute("""
        INSERT INTO parsed_resumes (
            person_name, resume_path, current_role,
            total_experience, current_tenure,
            certification_count, publication_count,
            skill_density, full_parsed_json,
            fitment_score, fitment_level, email
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        applicant_name,
        file,
        features.get('current_role'),
        features.get('total_experience'),
        features.get('current_tenure'),
        features.get('certification_count'),
        features.get('publication_count'),
        features.get('skill_density'),
        json.dumps(parsed),
        None,  # fitment_score (to be calculated later)
        None,  # fitment_level (to be classified later)
        email
    ))

conn.commit()
print(" All parsed resume data stored in Resumeparser.db.")


In [ ]:
import sqlite3
import pandas as pd

# --- Connect to the same database ---
db_path = "/content/sample_data/Resumeparser.db"
conn = sqlite3.connect(db_path)

# --- Check if 'parsed_resumes' table exists ---
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("📋 Available tables in DB:")
print(tables)

# --- View sample data if the table exists ---
if 'parsed_resumes' in tables['name'].values:
    df = pd.read_sql("SELECT * FROM parsed_resumes LIMIT 5", conn)
    print("\n📄 Sample Records from 'parsed_resumes':")
    print(df)
else:
    print("Table 'parsed_resumes' not found.")

conn.close()


In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("/content/sample_data/Resumeparser.db")

# Replace 'parsed_resumes' with your actual table name if different
table_name = "parsed_resumes"

# Get table schema
df_schema = pd.read_sql(f"PRAGMA table_info({table_name})", conn)

print(f"Columns in table '{table_name}':")
print(df_schema[['cid', 'name', 'type']])

print(f"\n Total fields: {len(df_schema)}")

conn.close()


In [ ]:
import os
import re
import json
import sqlite3
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import docx

# --- JD Path ---
jd_path = "/content/sample_data/Business Development Executive.docx"

# --- JD Text Extraction ---
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

# --- JD Feature Extraction ---
def extract_jd_features(jd_text):
    jd_text = jd_text.lower()
    target_role = jd_text.split('\n')[0].strip()
    cert_required = bool(re.search(r'\b(certification|certificate|certified)\b', jd_text))
    pub_required = bool(re.search(r'\b(publication|journal|conference|research)\b', jd_text))
    exp_match = re.search(r'(\d+)\s*(?:to|-)?\s*(\d*)\s*years', jd_text)
    min_exp = int(exp_match.group(1)) if exp_match else 0
    skill_keywords = [
        'python', 'sql', 'excel', 'communication', 'crm', 'presentation',
        'machine learning', 'ai', 'cloud', 'power bi', 'salesforce',
        'project', 'django', 'flask', 'deep learning'
    ]
    required_skills = [s for s in skill_keywords if s.lower() in jd_text]
    return {
        'target_role': target_role,
        'cert_required': cert_required,
        'pub_required': pub_required,
        'min_experience': min_exp,
        'expected_skills': required_skills
    }

# --- AHP Matrix ---
pairwise_matrix = np.array([
    [1,     4,     3,     5,     3,     4],     # experience
    [1/4,   1,     1/2,   2,     1/2,   1],     # certification
    [1/3,   2,     1,     3,     1,     2],     # tenure
    [1/5,   1/2,   1/3,   1,     1/3,   1],     # publication
    [1/3,   2,     1,     3,     1,     2],     # skill_density
    [1/4,   1,     1/2,   1,     1/2,   1]      # role_match
])

def compute_ahp_weights(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(eigvals.real)
    principal_eigvec = eigvecs[:, max_index].real
    weights = principal_eigvec / principal_eigvec.sum()
    return dict(zip(
        ['experience', 'certification', 'tenure', 'publication', 'skill_density', 'role_match'],
        np.round(weights, 4)
    ))

def compute_ahp_fitment_score(resume_features, jd_features, weights):
    resume_skills = set([s.lower() for s in resume_features.get('skills', [])]) if resume_features.get('skills') else set()
    expected_skills = set([s.lower() for s in jd_features.get('expected_skills', [])])
    S = len(resume_skills & expected_skills) / len(expected_skills) if expected_skills else 0
    C = 1 if jd_features['cert_required'] and resume_features['certification'] > 0 else 0
    P = 1 if jd_features['pub_required'] and resume_features['publication'] > 0 else 0
    E = min(resume_features['total_experience'] / jd_features['min_experience'], 1) if jd_features['min_experience'] > 0 else 1
    T = min(resume_features.get('current_tenure', 0) / 1, 1)
    R = SequenceMatcher(None, resume_features.get('current_role', '').lower(), jd_features.get('target_role', '').lower()).ratio()
    score = (
        weights['experience'] * E +
        weights['certification'] * C +
        weights['tenure'] * T +
        weights['publication'] * P +
        weights['skill_density'] * S +
        weights['role_match'] * R
    )
    return int(round(score * 100))

def classify_fit(score):
    if score < 31:
        return "Low Fit"
    elif score < 71:
        return "Medium Fit"
    else:
        return "Fit"

# --- Load JD and compute weights ---
jd_text = extract_text_from_docx(jd_path)
jd_features = extract_jd_features(jd_text)
# --- Show Extracted JD Features ---
print("\n Extracted JD Keywords and Features:")
for k, v in jd_features.items():
    print(f"{k}: {v}")

weights = compute_ahp_weights(pairwise_matrix)

# --- Connect to DB ---
conn = sqlite3.connect("/content/sample_data/Resumeparser.db")
cursor = conn.cursor()

# --- Read Data ---
df = pd.read_sql("SELECT rowid, * FROM parsed_resumes", conn)
print(f"Loaded {len(df)} resumes")

# --- Compute Scores and Update DB ---
for idx, row in df.iterrows():
    try:
        parsed = json.loads(row['full_parsed_json']) if isinstance(row['full_parsed_json'], str) else row['full_parsed_json']
    except:
        parsed = {}

    resume_features = {
        'current_role': row['current_role'],
        'total_experience': row['total_experience'],
        'current_tenure': row['current_tenure'],
        'certification': row['certification_count'],
        'publication': row['publication_count'],
        'skill_density': row['skill_density'],
        'skills': parsed.get("skills", [])
    }

    fitment_score = compute_ahp_fitment_score(resume_features, jd_features, weights)
    fitment_level = classify_fit(fitment_score)

    cursor.execute("""
        UPDATE parsed_resumes
        SET fitment_score = ?, fitment_level = ?
        WHERE rowid = ?
    """, (fitment_score, fitment_level, row['rowid']))

conn.commit()
conn.close()


In [ ]:
import numpy as np

# --- AHP Pairwise Matrix Example ---
pairwise_matrix = np.array([
    [1,     4,     3,     5,     3,     4],     # experience
    [1/4,   1,     1/2,   2,     1/2,   1],     # certification
    [1/3,   2,     1,     3,     1,     2],     # tenure
    [1/5,   1/2,   1/3,   1,     1/3,   1],     # publication
    [1/3,   2,     1,     3,     1,     2],     # skill_density
    [1/4,   1,     1/2,   1,     1/2,   1]      # role_match
])

criteria = ['experience', 'certification', 'tenure', 'publication', 'skill_density', 'role_match']

def compute_ahp_weights(matrix, labels):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(eigvals.real)
    principal_eigval = eigvals[max_index].real
    principal_eigvec = eigvecs[:, max_index].real

    normalized_weights = principal_eigvec / principal_eigvec.sum()

    print("Principal Eigenvalue (λmax):", round(principal_eigval, 4))
    print("\nNormalized AHP Weights:")
    for label, weight in zip(labels, normalized_weights):
        print(f"{label:<15}: {weight:.4f}")

    return dict(zip(labels, np.round(normalized_weights, 4)))

# Run the solver
weights = compute_ahp_weights(pairwise_matrix, criteria)


In [ ]:
import sqlite3
import pandas as pd

# --- Connect to your DB ---
conn = sqlite3.connect("/content/sample_data/Resumeparser.db")

# --- Read entire table ---
df = pd.read_sql("SELECT * FROM parsed_resumes", conn)

# --- Show first few rows ---
print("📄 Preview of parsed_resumes table:")
print(df.head())

# --- Show column names ---
print("\n Columns in table:")
print(df.columns.tolist())

# --- Optional: View summary ---
print("\n Row count:", len(df))

# Close connection
conn.close()


In [ ]:
print("\n Fitment Level Counts:")
print(df['fitment_level'].value_counts(dropna=False))

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to your SQLite DB
db_path = "/content/sample_data/Resumeparser.db"
conn = sqlite3.connect(db_path)

# Only select the columns that exist in your table
df = pd.read_sql("SELECT person_name, fitment_score FROM parsed_resumes", conn)
conn.close()

print(f"Total resumes loaded: {len(df)}")

# Plotting the histogram
plt.figure(figsize=(10, 6))
sns.histplot(df["fitment_score"], bins=10, kde=True, color="skyblue", edgecolor="black")

plt.title("Fitment Score Distribution of 148 Resumes for JD(Business Development Executive)", fontsize=14)
plt.xlabel("Fitment Score", fontsize=12)
plt.ylabel("Number of Resumes", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
print("\nFitment Score Stats:")
print(df['fitment_score'].describe())

# Optional: print how many scores are in different ranges
print("\n Score Range Buckets:")
print(df['fitment_score'].value_counts(bins=[0, 20, 40, 60, 80, 100], sort=False))


In [ ]:
!pip install python-docx matplotlib seaborn


In [ ]:
import os
import re
import json
import sqlite3
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import docx

# --- JD paths ---
jd_files = {
    "Business Development Executive": "/content/sample_data/Business Development Executive.docx",
    "Program Coordinator": "/content/sample_data/Program Coordinator.docx",
    "Research Associate": "/content/sample_data/Research Associate.docx"
}

# --- JD Text Extraction ---
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

# --- JD Feature Extraction ---
def extract_jd_features(jd_text):
    jd_text = jd_text.lower()
    target_role = jd_text.split('\n')[0].strip()
    cert_required = bool(re.search(r'\b(certification|certificate|certified)\b', jd_text))
    pub_required = bool(re.search(r'\b(publication|journal|conference|research)\b', jd_text))
    exp_match = re.search(r'(\d+)\s*(?:to|-)?\s*(\d*)\s*years', jd_text)
    min_exp = int(exp_match.group(1)) if exp_match else 0
    skill_keywords = [
        'python', 'sql', 'excel', 'communication', 'crm', 'presentation',
        'machine learning', 'ai', 'cloud', 'power bi', 'salesforce',
        'project', 'django', 'flask', 'deep learning'
    ]
    required_skills = [s for s in skill_keywords if s.lower() in jd_text]
    return {
        'target_role': target_role,
        'cert_required': cert_required,
        'pub_required': pub_required,
        'min_experience': min_exp,
        'expected_skills': required_skills
    }

# --- AHP ---
pairwise_matrix = np.array([
    [1,     4,     3,     5,     3,     4],
    [1/4,   1,     1/2,   2,     1/2,   1],
    [1/3,   2,     1,     3,     1,     2],
    [1/5,   1/2,   1/3,   1,     1/3,   1],
    [1/3,   2,     1,     3,     1,     2],
    [1/4,   1,     1/2,   1,     1/2,   1]
])

def compute_ahp_weights(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(eigvals.real)
    principal_eigvec = eigvecs[:, max_index].real
    weights = principal_eigvec / principal_eigvec.sum()
    return dict(zip(
        ['experience', 'certification', 'tenure', 'publication', 'skill_density', 'role_match'],
        np.round(weights, 4)
    ))

def compute_ahp_fitment_score(resume_features, jd_features, weights):
    resume_skills = set([s.lower() for s in resume_features.get('skills', [])]) if resume_features.get('skills') else set()
    expected_skills = set([s.lower() for s in jd_features.get('expected_skills', [])])
    S = len(resume_skills & expected_skills) / len(expected_skills) if expected_skills else 0
    C = 1 if jd_features['cert_required'] and resume_features['certification'] > 0 else 0
    P = 1 if jd_features['pub_required'] and resume_features['publication'] > 0 else 0
    E = min(resume_features['total_experience'] / jd_features['min_experience'], 1) if jd_features['min_experience'] > 0 else 1
    T = min(resume_features.get('current_tenure', 0) / 1, 1)
    R = SequenceMatcher(None, resume_features.get('current_role', '').lower(), jd_features.get('target_role', '').lower()).ratio()
    score = (
        weights['experience'] * E +
        weights['certification'] * C +
        weights['tenure'] * T +
        weights['publication'] * P +
        weights['skill_density'] * S +
        weights['role_match'] * R
    )
    return int(round(score * 100))

# --- Load Resumes from DB ---
conn = sqlite3.connect("/content/sample_data/Resumeparser.db")
df_all = pd.read_sql("SELECT * FROM parsed_resumes", conn)

# --- Compute Scores for Each JD ---
weights = compute_ahp_weights(pairwise_matrix)
jd_dfs = []

for jd_name, jd_path in jd_files.items():
    jd_text = extract_text_from_docx(jd_path)
    jd_features = extract_jd_features(jd_text)

    temp_df = df_all.copy()
    scores = []
    for _, row in temp_df.iterrows():
        try:
            parsed = json.loads(row['full_parsed_json']) if isinstance(row['full_parsed_json'], str) else {}
        except:
            parsed = {}
        resume_features = {
            'current_role': row['current_role'],
            'total_experience': row['total_experience'],
            'current_tenure': row['current_tenure'],
            'certification': row['certification_count'],
            'publication': row['publication_count'],
            'skill_density': row['skill_density'],
            'skills': parsed.get("skills", [])
        }
        score = compute_ahp_fitment_score(resume_features, jd_features, weights)
        scores.append(score)

    temp_df["fitment_score"] = scores
    temp_df["JD"] = jd_name
    jd_dfs.append(temp_df[["person_name", "skill_density", "fitment_score", "JD"]])

conn.close()

# Final merged DataFrame
combined_df = pd.concat(jd_dfs, ignore_index=True)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Bin skill_density for smoother lines (optional)
combined_df['skill_bin'] = pd.cut(combined_df['skill_density'], bins=10)

# Group by JD and skill_bin to get mean fitment_score
line_data = (
    combined_df
    .groupby(['JD', 'skill_bin'], observed=True)['fitment_score']
    .mean()
    .reset_index()
)

# Convert bin labels to mid-points for plotting
line_data['skill_bin_mid'] = line_data['skill_bin'].apply(lambda x: x.mid)

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=line_data,
    x='skill_bin_mid',
    y='fitment_score',
    hue='JD',
    palette=["#003f5c", "#ffa600", "#7f4f24"],  # dark blue, orange, brown
    linewidth=2.5,
    marker='o'
)

plt.title("Average AHP Fitment Score by Skill Density", fontsize=14)
plt.xlabel("Skill Density", fontsize=12)
plt.ylabel("Fitment Score (AHP)", fontsize=12)
plt.grid(True)
plt.legend(title="Job Description")
plt.tight_layout()
plt.show()


In [ ]:
import sqlite3
import pandas as pd

# Load full parsed resumes from DB
conn = sqlite3.connect("/content/sample_data/Resumeparser.db")
df_all = pd.read_sql("SELECT * FROM parsed_resumes", conn)
conn.close()

# Work on full resume data
df = df_all.copy()

# Features to normalize
expected_columns = [
    "total_experience",
    "current_tenure",
    "certification_count",
    "publication_count",
    "skill_density"
]

# Check for missing columns
missing_cols = [col for col in expected_columns if col not in df.columns]
if missing_cols:
    raise KeyError(f" Missing columns in df: {missing_cols}")

# Binary normalize each feature
for col in expected_columns:
    max_val = df[col].max()
    if max_val > 0:
        df[f"norm_{col}"] = (df[col] / max_val >= 0.5).astype(int)
    else:
        df[f"norm_{col}"] = 0

# Preview results
print("\n Normalized Columns Preview:")
print(df[[f"norm_{col}" for col in expected_columns]].head())


In [ ]:
!pip install -U shiny

In [ ]:
!pip install numpy==1.26.4


In [ ]:
import shiny
print(shiny.__version__)


In [ ]:
#  Restart and clean old processes
!pkill -f shiny
!pkill -f cloudflared
#   shiny_code = r"""

In [ ]:
%%writefile app_config.py
import sqlite3
import ahpy

# Database Path
DB_PATH = "/content/sample_data/Resumeparser.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)

#  Keywords and Role Helpers
irrelevant_keywords = [
    'home town', 'education certifications', 'certifications', 'certification', 'education', 'summary',
    'objective', 'personal', 'contact', 'details', 'profile', 'curriculum', 'vitae', 'resume', 'skills',
    'hobbies', 'interests', 'languages', 'declaration', 'reference', 'references', 'address', 'email',
    'phone', 'mobile', 'dob', 'date of birth'
]




job_keywords = [
    "engineer", "developer", "manager", "analyst", "consultant", "specialist", "coordinator",
    "executive", "associate", "administrator", "advisor", "officer", "lead", "head", "director",
    "scientist", "architect", "designer", "supervisor", "technician", "trainer", "assistant"
]

fallback_titles = ["Engineer", "Analyst", "Developer", "Consultant"]

#  Skills to scan from resumes & JDs
skill_keywords = ['python', 'sql', 'excel', 'communication', 'crm', 'presentation', 'ai', 'cloud']

# AHP Pairwise Comparison Matrix
pairwise = {
    ('experience', 'certification'): 3,
    ('experience', 'tenure'): 2,
    ('experience', 'skill_density'): 2,
    ('experience', 'publication'): 4,
    ('experience', 'role_match'): 2,
    ('certification', 'tenure'): 1,
    ('certification', 'skill_density'): 0.5,
    ('certification', 'publication'): 2,
    ('certification', 'role_match'): 1,
    ('tenure', 'skill_density'): 0.5,
    ('tenure', 'publication'): 2,
    ('tenure', 'role_match'): 1,
    ('skill_density', 'publication'): 3,
    ('skill_density', 'role_match'): 1,
    ('publication', 'role_match'): 0.5,
}

criteria = ahpy.Compare("Fitment", pairwise)
default_ahp_weights = criteria.target_weights.copy()  # Used when no manual slider input

#  UI Column Mapping
col_map = {
    "person_name": "Name",
    "current_role": "Current Role",
    "total_experience": "Experience",
    "current_tenure": "Tenure",
    "certification_count": "Certifications",
    "publication_count": "Publications",
    "skill_density": "Skills",
    "fitment_score": "Score",
    "fitment_level": "Level",
}

# These will be normalized later (binary 0/1)
features_to_normalize = [
    "total_experience",
    "current_tenure",
    "certification_count",
    "publication_count",
    "skill_density"
]


Writing app_config.py


In [ ]:
%%writefile extractors.py
import os
import re
import fitz
import docx
import random
import string
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyresparser import ResumeParser
from difflib import SequenceMatcher
from pdf2image import convert_from_path
import pytesseract

from app_config import irrelevant_keywords, job_keywords, fallback_titles, skill_keywords, pairwise

# ---------- Utility Functions ----------

invalid_role_keywords = [
    # Degrees/fields
    'b.tech', 'btech', 'b.e', 'be', 'bachelor', 'master', 'msc', 'm.sc', 'mba', 'pgdm', 'phd',
    'engineering', 'computer science', 'information technology', 'electronics', 'civil', 'mechanical',
    # Section headers
    'education', 'skills', 'projects', 'experience', 'certifications', 'achievements', 'summary',
    'objective', 'profile', 'about me', 'contact', 'personal details',
    # Soft skills/qualities
    'leadership', 'leadership qualities', 'leadership quality', 'communication', 'team player',
    'problem solving', 'quick learner', 'hardworking', 'presentation skills', 'good communication',
    # Formatting artifacts
    '•', '', '', '-', '_', '—', '–', '•', '·', '…', '“', '”', '‘', '’',
    # Company names (add your own if needed)
    'leadsquared',
    # Misc
    'simply', 'trainee', 'intern', 'fresher','home town', 'education certifications', 'certifications', 'certification', 'education', 'summary',
    'objective', 'personal', 'contact', 'details', 'profile', 'curriculum', 'vitae', 'resume', 'skills',
    'hobbies', 'interests', 'languages', 'declaration', 'reference', 'references', 'address', 'email',
    'phone', 'mobile', 'dob', 'date of birth'
]

def extract_text_with_ocr(pdf_path):
    try:
        pages = convert_from_path(pdf_path)
        return "\n".join([pytesseract.image_to_string(page) for page in pages])
    except Exception as e:
        print(f" OCR failed for {pdf_path}: {e}")
        return ""

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            text = "".join([page.get_text() for page in doc])
    except Exception as e:
        print(f" Error reading {pdf_path}: {e}")
    return text.strip() or extract_text_with_ocr(pdf_path)

def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

# ---------- JD Processing ----------
def extract_jd_features(jd_text):
    jd_text = jd_text.lower()
    target_role = jd_text.split('\n')[0].strip()
    cert_required = bool(re.search(r'\b(certification|certificate|certified)\b', jd_text))
    pub_required = bool(re.search(r'\b(publication|journal|conference|research)\b', jd_text))
    exp_match = re.search(r'(\d+)\s*(?:to|-)?\s*(\d*)\s*years', jd_text)
    min_exp = int(exp_match.group(1)) if exp_match else 0
    required_skills = [s for s in skill_keywords if s.lower() in jd_text]
    return {
        'target_role': target_role,
        'cert_required': cert_required,
        'pub_required': pub_required,
        'min_experience': min_exp,
        'expected_skills': required_skills
    }

def process_jd_file(jd_path):
    jd_text = extract_text_from_docx(jd_path)
    jd_features = extract_jd_features(jd_text)
    ahp_weights = compute_ahp_weights(pairwise_matrix)
    return jd_text, jd_features, ahp_weights

# ---------- AHP Weight Calculation ----------
pairwise_matrix = np.array([
    [1,     4,     3,     5,     3,     4],
    [1/4,   1,     1/2,   2,     1/2,   1],
    [1/3,   2,     1,     3,     1,     2],
    [1/5,   1/2,   1/3,   1,     1/3,   1],
    [1/3,   2,     1,     3,     1,     2],
    [1/4,   1,     1/2,   1,     1/2,   1]
])

def compute_ahp_weights(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(eigvals.real)
    principal_eigvec = eigvecs[:, max_index].real
    weights = principal_eigvec / principal_eigvec.sum()
    return dict(zip(
        ['experience', 'certification', 'tenure', 'publication', 'skill_density', 'role_match'],
        np.round(weights, 4)
    ))

# ---------- Role Extraction (Same as Before) ----------
def is_irrelevant_role(role):
    if not role:
        return True
    role_lower = role.lower().strip()
    if role_lower in irrelevant_keywords:
        return True
    if len(role_lower) < 3 or len(role_lower) > 40:
        return True
    if re.fullmatch(r'\d+', role_lower):
        return True
    if not any(k in role_lower for k in job_keywords):
        return True
    return False

def clean_designation(role):
    if not role or not isinstance(role, str):
        return ""
    role = re.sub(r'\b(consultant|intern|trainee|project|fresher|certified|course|college|university|student|degree|btech|engineering)\b', '', role, flags=re.I)
    role = re.sub(r'[-@|/]\s*\w+$', '', role)
    role = re.sub(r'\b(and|with|for|in|of|from)$', '', role.strip(), flags=re.I)
    role = re.sub(r'[^a-zA-Z\s\-]', '', role)
    role = re.sub(r'\s+', ' ', role).strip()
    words = role.split()
    if 1 <= len(words) <= 5:
        return role.title()
    else:
        return ""

def guess_role_from_lines(text):
    for line in text.splitlines():
        line = line.strip()
        if not (1 <= len(line.split()) <= 5):
            continue
        if any(k in line.lower() for k in job_keywords):
            return line.title()
    return random.choice(fallback_titles)

def extract_most_recent_role(text):
    date_pattern = re.compile(
        r'((?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})\s*[-–to]+\s*(present|current|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})',
        re.IGNORECASE
    )
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    for i, line in enumerate(lines):
        if date_pattern.search(line):
            for j in range(1, 3):
                if i - j >= 0:
                    candidate = lines[i - j]
                    for part in re.split(r'\||,| and |/| at ', candidate):
                        part = part.strip()
                        if is_valid_role(part):
                            return part.title()
    for line in lines:
        for part in re.split(r'\||,| and |/| at ', line):
            part = part.strip()
            if is_valid_role(part):
                return part.title()
    return random.choice(fallback_titles)

def is_valid_role(line):
    l = line.lower().strip()
    l = l.strip(string.punctuation + "•-—–·…“”‘’")
    # Exclude lines with academic titles or references
    if any(x in l for x in ['advisor:', 'prof.', 'dr.', 'reference', 'referee', 'guide', 'supervisor']):
        return False
    if not (1 <= len(l.split()) <= 6):
        return False
    if any(char.isdigit() for char in l) or "http" in l or "www" in l:
        return False
    for kw in irrelevant_keywords:
        if kw in l:
            return False
    if l.isupper() or l.split()[0] in ['education', 'skills', 'projects', 'experience', 'summary', 'objective', 'profile']:
        return False
    if any(jk in l for jk in job_keywords):
        return True
    return False

def guess_role_from_text(text):
    for line in text.splitlines():
        l = line.strip().lower()
        if not (1 <= len(l.split()) <= 3):
            continue
        if any(char.isdigit() for char in l):
            continue
        if any(k in l for k in invalid_role_keywords):
            continue
        if any(jk in l for jk in job_keywords):
            return line.strip().title()
    return random.choice(fallback_titles)

def extract_designation_from_text(text):
    lines = [line.strip() for line in text.splitlines()]
    in_experience = False
    for line in lines:
        if "professional experience" in line.lower():
            in_experience = True
            continue
        if in_experience and line:
            if len(line.split()) <= 5 and not any(x in line.lower() for x in ["january", "february", "2023", "2022", "present", "company", "location", "@"]):
                return line.strip()
    return ""


# ---------- Resume Feature Extraction ----------
def extract_resume_features_combined(file_path, resume_text):
    lowered = resume_text.lower()
    lines = resume_text.splitlines()
    date_pattern = re.compile(
        r'((?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})\s*[-–to]+\s*(present|current|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[\s\-.,]*\d{2,4}|\d{1,2}/\d{4})',
        re.IGNORECASE
    )

    total_months = 0
    latest_start = None
    job_count = 0

    for line in lines:
        match = date_pattern.search(line)
        if match:
            try:
                start_str, end_str = match.groups()
                start_date = datetime.strptime(start_str.strip()[:3] + start_str.strip()[-4:], '%b%Y') if '/' not in start_str else datetime.strptime(start_str.strip(), '%m/%Y')
                end_date = datetime.now() if 'present' in end_str.lower() else datetime.strptime(end_str.strip()[:3] + end_str.strip()[-4:], '%b%Y') if '/' not in end_str else datetime.strptime(end_str.strip(), '%m/%Y')
                if start_date <= end_date:
                    delta = relativedelta(end_date, start_date)
                    total_months += delta.years * 12 + delta.months
                    job_count += 1
                    if latest_start is None or start_date > latest_start:
                        latest_start = start_date
            except:
                continue

    current_tenure_months = 0
    if latest_start:
        delta = relativedelta(datetime.now(), latest_start)
        current_tenure_months = delta.years * 12 + delta.months

    total_exp = int(round(total_months / 12))
    current_tenure = int(round(current_tenure_months / 12))

    current_role = ""
    skills = []
    parser_exp = 0

    try:
        parsed = ResumeParser(file_path).get_extracted_data()
        raw_role = parsed.get('designation', "")
        if isinstance(raw_role, list):
            raw_role = raw_role[0] if raw_role else ""
        role_from_parser = clean_current_role(raw_role, resume_text)
        skills = parsed.get('skills', [])
        parser_exp = parsed.get('total_experience', 0) or 0
    except Exception as e:
        print(f" ResumeParser failed for {os.path.basename(file_path)}: {e}")
        role_from_parser = ""
        skills = []

    role_from_context = extract_most_recent_role(resume_text)
    role_from_guess = guess_role_from_text(resume_text)

    for candidate in [role_from_parser, role_from_context, role_from_guess]:
        if candidate and is_valid_role(candidate):
            current_role = candidate
            break
    else:
        current_role = random.choice(fallback_titles)

    if total_exp == 0 and current_tenure == 0 and parser_exp:
        total_exp = int(round(parser_exp))
        current_tenure = total_exp

    if total_exp == 0 and current_tenure > 0:
        total_exp = current_tenure
    if current_tenure == 0 and total_exp > 0:
        current_tenure = total_exp
    if current_tenure > total_exp:
        current_tenure = total_exp

    exp_phrase = re.search(r'(\d+)\s+years? of experience', lowered)
    if total_exp == 0 and current_tenure == 0 and exp_phrase:
        total_exp = int(exp_phrase.group(1))
        current_tenure = total_exp

    certs = len(re.findall(r'\b(certification|certificate|certified)\b', lowered))
    pubs = len(re.findall(r'\b(publication|journal|conference|research)\b', lowered))
    density = sum(lowered.count(skill) for skill in skill_keywords)

    return {
        'current_role': current_role,
        'total_experience': total_exp,
        'current_tenure': current_tenure,
        'certification_count': certs,
        'publication_count': pubs,
        'skill_density': density,
        'job_count': job_count,
        'skills': skills
    }

# ---------- Person Name Extraction ----------
blacklist = {
    "curriculum", "resume", "btech", "b.e", "be", "mba", "mca", "mtech", "cse", "Curriculum Vitae","Puc ","Driven Decision","Email","Education Post",
    "ece", "eee","internal","experience","Krishna University","Internal Experience	","Pgdm E","Nga Vas","Relationship Building","Skills ",
    "civil", "mech", "ece", "student", "intern", "fresher", "developer", "engineer", "job","university","woolf","University Of Pune",
    "Tamil Nadu","Cyber Security","Microchip Company","J A","Hackerank Httpswww Hackerrank Rakeshtm	"," Business","Tamil Nadu","Newresume",
    "file", "document", "doc", "new", "final", "latest", "updated", "resume1", "resume2","Navi Mumbai","Threat Analysis","Race Job",
    "Pre University College","C O","Academic Counsellor","Documentation Presentation",
}

def extract_person_name(resume_text, resume_path=None, email=None, parsed_name=None):
    def clean_name(name):
        name = name.lower().strip()
        name = re.sub(r'[^a-z ]+', '', name)
        name = re.sub(r'\s+', ' ', name)
        return name.title()

    if parsed_name:
        name = clean_name(parsed_name)
        if name.lower() not in blacklist and 1 <= len(name.split()) <= 4:
            return name

    if email:
        username = email.split("@")[0]
        name = clean_name(username.replace(".", " ").replace("_", " ").replace("-", " "))
        parts = [p for p in name.split() if p.lower() not in blacklist and not any(char.isdigit() for char in p)]
        if 1 <= len(parts) <= 3:
            return " ".join([p.title() for p in parts])

    # NEW LOGIC: Look for a plausible name in the first 10 lines of resume
    header_lines = resume_text.strip().split('\n')[:10]
    for line in header_lines:
        name = clean_name(line)
        if name.lower() not in blacklist and 1 <= len(name.split()) <= 4 and len(name) > 2:
            return name

    return "Unknown"




# ---------- AHP Fitment Score Calculation ----------
def compute_ahp_fitment_score(resume_features, jd_features, weights):
    resume_skills = set([s.lower() for s in resume_features.get('skills', [])])
    expected_skills = set([s.lower() for s in jd_features.get('expected_skills', [])])
    S = len(resume_skills & expected_skills) / len(expected_skills) if expected_skills else 0
    C = 1 if jd_features['cert_required'] and resume_features['certification_count'] > 0 else 0
    P = 1 if jd_features['pub_required'] and resume_features['publication_count'] > 0 else 0
    E = min(resume_features['total_experience'] / jd_features['min_experience'], 1) if jd_features['min_experience'] > 0 else 1
    T = min(resume_features.get('current_tenure', 0) / 1, 1)
    R = SequenceMatcher(None, resume_features.get('current_role', '').lower(), jd_features.get('target_role', '').lower()).ratio()
    score = (
        weights['experience'] * E +
        weights['certification'] * C +
        weights['tenure'] * T +
        weights['publication'] * P +
        weights['skill_density'] * S +
        weights['role_match'] * R
    )
    return int(round(score * 100))

def classify_fit(score):
    if score < 31:
        return "Low Fit"
    elif score < 71:
        return "Medium Fit"
    else:
        return "Fit"


Writing extractors.py


In [ ]:
%%writefile db_utils.py
from app_config import conn

def save_to_db(data):
    cursor = conn.cursor()
    for row in data:
        # Normalize keys to match DB column names
        row["certification_count"] = row.pop("certifications", 0)
        row["publication_count"] = row.pop("publications", 0)

        cursor.execute('''
        UPDATE parsed_resumes
        SET
            person_name = ?,
            current_role = ?,
            total_experience = ?,
            current_tenure = ?,
            certification_count = ?,
            publication_count = ?,
            skill_density = ?,
            fitment_score = ?,
            fitment_level = ?
        WHERE LOWER(TRIM(REPLACE(resume_path, '\\', '/'))) LIKE ?
        ''', (
            row["person_name"], row["current_role"],
            row["total_experience"], row["current_tenure"],
            row["certification_count"], row["publication_count"], row["skill_density"],
            row["fitment_score"], row["fitment_level"],
            f'%/{row["file_name"].lower()}'
        ))
    conn.commit()

def load_from_db(filenames):
    import pandas as pd
    cursor = conn.cursor()
    placeholders = ','.join(['?'] * len(filenames))
    normalized_filenames = [f.lower() for f in filenames]

    if len(filenames) == 1:
        query = """
        SELECT * FROM parsed_resumes
        WHERE LOWER(REPLACE(resume_path, '\\\\', '/')) LIKE ?
        """
        params = [f'%/{normalized_filenames[0]}']
    else:
        conditions = " OR ".join(
            ["LOWER(REPLACE(resume_path, '\\\\', '/')) LIKE ?" for _ in filenames]
        )
        query = f"""
        SELECT * FROM parsed_resumes
        WHERE {conditions}
        """
        params = [f'%/{f}' for f in normalized_filenames]

    df = pd.read_sql_query(query, conn, params=params)
    return df



Writing db_utils.py


In [ ]:
%%writefile ui_layout.py
from shiny import ui
from app_config import col_map

app_ui = ui.page_fluid(
    ui.navset_tab(
        ui.nav_panel("📊 Dashboard",
            ui.output_ui("dashboard_text_summary"),
            ui.h4("Shortlisted Candidates"),
            ui.layout_columns(
                ui.div(
                    ui.input_checkbox_group(
                        "shortlist_columns",
                        "Columns to show:",
                        choices=col_map,
                        selected=["person_name", "fitment_score"]
                    ),
                    ui.download_button("download_shortlist", "Download Shortlist as CSV"),
                    style="margin-bottom: 1em;"
                ),
                ui.div(
                    ui.output_data_frame("shortlist_table"),
                    style="width:100%"
                ),
                col_widths=[3, 9]
            )
        ),
        ui.nav_menu("🗃️ Data Mart",
            ui.nav_panel("Data Upload",
                ui.input_file("jd_upload", "Upload JDs (.docx)", multiple=True, accept=[".docx"]),
                ui.input_file("resume_upload", "Upload Resumes (.pdf, .docx)", multiple=True, accept=[".pdf", ".docx"]),
                ui.output_text_verbatim("upload_status"),
                ui.output_ui("jd_selector_view")
            ),
            ui.nav_panel("Data View",
                ui.output_data_frame("dataview_table")
            )
        ),
        ui.nav_menu("⚖️ Fitment",
            ui.nav_panel("Prioritization (AHP)",
                ui.layout_sidebar(
                    ui.sidebar(
                        ui.h4("Adjust AHP Weights"),
                        ui.input_slider("weight_experience", "Experience Weight", 0, 10, 5),
                        ui.input_slider("weight_skill", "Skill Weight", 0, 10, 5),
                        ui.input_slider("weight_tenure", "Tenure Weight", 0, 10, 5),
                        ui.input_slider("weight_cert", "Certification Weight", 0, 10, 5),
                        ui.input_slider("weight_pub", "Publication Weight", 0, 10, 5),
                        ui.input_slider("weight_role", "Role Match Weight", 0, 10, 5),
                        ui.input_action_button("apply_weights", "Apply Weights"),
                        ui.input_action_button("reset_weights", "Reset Weights", class_="btn-danger")
                    ),
                    ui.output_data_frame("fitment_ahp_table")
                )
            ),
            ui.nav_panel("Parser",
                ui.h4("Extracted Resume Data"),
                ui.output_data_frame("parser_table")
            ),
            ui.nav_panel("Fitment Score",
                ui.input_select("fitment_filter_score", "Filter by Fitment:", {
                    "All": "All",
                    "Low Fit": "Low Fit",
                    "Medium Fit": "Medium Fit",
                    "Fit": "Fit"
                }, selected="All"),
                ui.output_data_frame("fitment_score_table")
            )
        )
    )
)


Writing ui_layout.py


In [ ]:
%%writefile main_app.py
import os
import sqlite3
import pandas as pd
import tempfile

from shiny import App, reactive, render, ui

from app_config import default_ahp_weights, col_map
from ui_layout import app_ui
from extractors import (
    extract_text_from_docx, extract_text_from_pdf,
    extract_resume_features_combined as extract_resume_features,
    extract_jd_features, compute_ahp_fitment_score as compute_ahp_score,
    classify_fit, extract_person_name
)

from db_utils import save_to_db

DB_PATH = "/content/sample_data/Resumeparser.db"

jd_store = reactive.Value([])
resume_store = reactive.Value([])
ahp_weights = reactive.Value(default_ahp_weights.copy())
features_to_normalize = [
    "total_experience", "current_tenure",
    "certification_count", "publication_count", "skill_density"
]

def fetch_features_from_db(filename):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query(
        "SELECT * FROM parsed_resumes WHERE resume_path LIKE ?",
        conn, params=(f"%{filename}",)
    )
    conn.close()
    return df.iloc[0].to_dict() if not df.empty else None

def normalize_df(df):
    for col in features_to_normalize:
        max_val = df[col].max()
        if max_val > 0:
            df[f"norm_{col}"] = (df[col] / max_val >= 0.5).astype(int)
        else:
            df[f"norm_{col}"] = 0
    return df

def server(input, output, session):

    @reactive.Effect
    def _():
        if input.jd_upload():
            jd_store.set(input.jd_upload())
        if input.resume_upload():
            resume_store.set(input.resume_upload())

    @output
    @render.text
    def upload_status():
        return f"{len(jd_store.get())} JD(s) uploaded, {len(resume_store.get())} resume(s) uploaded"

    @output
    @render.ui
    def jd_selector_view():
        jds = jd_store.get()
        choices = {f["name"]: f["name"] for f in jds} if jds else {}
        return ui.input_select("jd_select_view", "Select JD:", choices=choices)

    @reactive.Calc
    def selected_jd_features():
        jds = jd_store.get()
        selected_name = input.jd_select_view()
        if not selected_name:
            return None
        jd_file = next((jd for jd in jds if jd["name"] == selected_name), None)
        if not jd_file:
            return None
        with tempfile.NamedTemporaryFile(delete=False, suffix=".docx") as tmp_jd:
            with open(jd_file["datapath"], "rb") as f:
                tmp_jd.write(f.read())
            jd_text = extract_text_from_docx(tmp_jd.name)
            return extract_jd_features(jd_text)

    @reactive.Calc
    def dashboard_results():
        resumes = resume_store.get()
        jd_features = selected_jd_features()
        if not resumes or not jd_features:
            df = pd.DataFrame()
            # But still save empty as backup for robustness
            df.to_csv("dashboard_shortlist_latest.csv", index=False)
            return df

        weights = ahp_weights.get()
        rows = []
        for resume in resumes:
            filename = resume["name"].lower()
            db_features = fetch_features_from_db(filename)
            if db_features:
                person_name = db_features.get("person_name") or os.path.splitext(filename)[0]
                score = compute_ahp_score(db_features, jd_features, weights)
                fit = classify_fit(score)
                row = {
                    **db_features,
                    "file_name": filename,
                    "person_name": person_name,
                    "fitment_score": score,
                    "fitment_level": fit,
                }
            else:
                ext = os.path.splitext(resume["name"])[-1].lower()
                suffix = ".pdf" if ext == ".pdf" else ".docx"
                with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
                    with open(resume["datapath"], "rb") as f:
                        tmp.write(f.read())
                    text = extract_text_from_pdf(tmp.name) if suffix == ".pdf" else extract_text_from_docx(tmp.name)
                    features = extract_resume_features(tmp.name, text)
                    person_name = extract_person_name(resume_text=text, resume_path=tmp.name)
                    if not person_name or person_name.lower() == "unknown":
                        # fallback to file name
                        person_name = os.path.splitext(filename)[0].replace("_", " ").title()
                    score = compute_ahp_score(features, jd_features, weights)
                    fit = classify_fit(score)
                    row = {
                        **features,
                        "file_name": filename,
                        "person_name": person_name,
                        "fitment_score": score,
                        "fitment_level": fit,
                    }
            rows.append(row)
        df = pd.DataFrame(rows)
        for col in features_to_normalize:
            max_val = df[col].max() if not df.empty else 0
            if max_val > 0:
                df[f"norm_{col}"] = (df[col] / max_val >= 0.5).astype(int)
            else:
                df[f"norm_{col}"] = 0

        # Save exact shortlist currently viewing on dashboard in CSV as backup
        try:
            df.to_csv("dashboard_shortlist_latest.csv", index=False)
        except Exception as e:
            print("Could not save current shortlist as CSV:", e)

        #  Save updated rows to DB
        try:
            save_to_db(rows)
        except Exception as e:
            print("Could not save updated rows to DB:", e)

        return df

    @output
    @render.ui
    def dashboard_text_summary():
        df = dashboard_results()
        if df.empty:
            return ui.pre("No data available.")

        return ui.div(
            ui.pre(
                f"Total Resumes: {len(df)}\n"
                f"Experience > 1 year: {df['norm_total_experience'].sum()}\n"
                f"Tenure > 1 year: {df['norm_current_tenure'].sum()}\n"
                f"Certifications present: {df['norm_certification_count'].sum()}\n"
                f"Publications present: {df['norm_publication_count'].sum()}\n"
                f"Skill keywords present: {df['norm_skill_density'].sum()}\n"
                f"Shortlisted (Fit): {(df['fitment_level'] == 'Fit').sum()}"
            ),
            ui.input_radio_buttons(
                "shortlist_filter_option",
                "Show Candidates by:",
                {
                    "All": "All", "Experience": "Experience", "Tenure": "Tenure",
                    "Certification": "Certification", "Publication": "Publication", "Skills": "Skills"
                },
                selected="All", inline=True
            )
        )

    @output
    @render.data_frame
    def shortlist_table():
        df = dashboard_results()
        if df.empty:
            return pd.DataFrame()
        option = input.shortlist_filter_option()
        if option == "Experience":
            df = df[df["norm_total_experience"] == 1]
        elif option == "Tenure":
            df = df[df["norm_current_tenure"] == 1]
        elif option == "Certification":
            df = df[df["norm_certification_count"] == 1]
        elif option == "Publication":
            df = df[df["norm_publication_count"] == 1]
        elif option == "Skills":
            df = df[df["norm_skill_density"] == 1]
        selected_cols = input.shortlist_columns() or ["person_name", "fitment_score"]
        return df[[col for col in selected_cols if col in df.columns]]

    @output
    @render.download(filename="dashboard_shortlist.csv")
    def download_shortlist():
        # Download exactly what is currently showing in shortlist_table
        def writer(file):
            try:
                df = dashboard_results()
                if df.empty:
                    file.write("No data available.")
                    return

                # Apply same filter as shortlist_table
                option = input.shortlist_filter_option()
                if option == "Experience":
                    df = df[df["norm_total_experience"] == 1]
                elif option == "Tenure":
                    df = df[df["norm_current_tenure"] == 1]
                elif option == "Certification":
                    df = df[df["norm_certification_count"] == 1]
                elif option == "Publication":
                    df = df[df["norm_publication_count"] == 1]
                elif option == "Skills":
                    df = df[df["norm_skill_density"] == 1]

                selected_cols = input.shortlist_columns() or ["person_name", "fitment_score"]
                filtered_df = df[[col for col in selected_cols if col in df.columns]].copy()
                # Optional pretty col names
                rename_map = {col: col_map.get(col, col) for col in filtered_df.columns}
                filtered_df.rename(columns=rename_map, inplace=True)
                # Write actual CSV
                filtered_df.to_csv(file, index=False)
            except Exception as e:
                import traceback
                traceback.print_exc()
                file.write("Download failed due to an internal error.")
        return writer

    @reactive.Calc
    def dataview_table_data():
        resumes = resume_store.get()
        jd_features = selected_jd_features()
        if not resumes or not jd_features:
            return pd.DataFrame(columns=[
                "file_name", "person_name", "current_role",
                "norm_total_experience", "norm_current_tenure",
                "norm_certification_count", "norm_publication_count",
                "norm_skill_density", "fitment_score", "fitment_level"
            ])
        weights = ahp_weights.get()
        results = []
        for resume in resumes:
            filename = resume["name"]
            db_features = fetch_features_from_db(filename)
            if db_features:
                # Known/resume in DB
                score = compute_ahp_score(db_features, jd_features, weights)
                fit = classify_fit(score)
                row = {
                    **db_features,
                    "file_name": filename,
                    "fitment_score": score,
                    "fitment_level": fit,
                }
                # Ensure person_name is present, fallback to filename
                if not row.get("person_name") or row["person_name"].lower() == "unknown":
                    row["person_name"] = os.path.splitext(filename)[0].replace("_", " ").title()
                results.append(row)
            else:
                # New resume: extract, score, save, show
                ext = os.path.splitext(filename)[-1].lower()
                suffix = ".pdf" if ext == ".pdf" else ".docx"
                with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
                    with open(resume["datapath"], "rb") as f:
                        tmp.write(f.read())
                    text = extract_text_from_pdf(tmp.name) if suffix == ".pdf" else extract_text_from_docx(tmp.name)
                    features = extract_resume_features(tmp.name, text)
                    # Extract person_name from resume text, fallback to filename
                    person_name = extract_person_name(resume_text=text)
                    if not person_name or person_name.lower() == "unknown":
                        # fallback
                        person_name = os.path.splitext(filename)[0].replace("_", " ").title()

                    score = compute_ahp_score(features, jd_features, weights)
                    fit = classify_fit(score)
                    row = {
                        **features,
                        "file_name": filename,
                        "person_name": person_name,
                        "fitment_score": score,
                        "fitment_level": fit,
                    }
                    # Save for DB so future runs are fast
                    save_to_db([row])
                    results.append(row)
        # Build DataFrame and normalize
        df = pd.DataFrame(results)
        for col in features_to_normalize:
            max_val = df[col].max() if not df.empty else 0
            if max_val > 0:
                df[f"norm_{col}"] = (df[col] / max_val >= 0.5).astype(int)
            else:
                df[f"norm_{col}"] = 0
        # Always return DF with all columns present
        columns_out = [
            "file_name", "person_name", "current_role",
            "norm_total_experience", "norm_current_tenure",
            "norm_certification_count", "norm_publication_count",
            "norm_skill_density", "fitment_score", "fitment_level"
        ]
        # Add any missing columns as empty/zeros
        for col in columns_out:
            if col not in df.columns:
                df[col] = ""
        return df[columns_out]

    @output
    @render.data_frame
    def dataview_table():
        df = dataview_table_data()
        if df.empty:
            return pd.DataFrame(columns=[
                "file_name", "person_name", "current_role",
                "norm_total_experience", "norm_current_tenure",
                "norm_certification_count", "norm_publication_count",
                "norm_skill_density", "fitment_score", "fitment_level"
            ])
        return df[[
            "file_name", "person_name", "current_role",
            "norm_total_experience", "norm_current_tenure",
            "norm_certification_count", "norm_publication_count",
            "norm_skill_density", "fitment_score", "fitment_level"
        ]]


    @output
    @render.data_frame
    def fitment_ahp_table():
        df = dashboard_results()
        return df[["person_name", "fitment_score", "fitment_level"]] if not df.empty else pd.DataFrame()

    @reactive.Effect
    def _():
        if input.apply_weights():
            total = (
                input.weight_experience() + input.weight_skill() +
                input.weight_tenure() + input.weight_cert() +
                input.weight_pub() + input.weight_role()
            )
            if total > 0:
                new_weights = {
                    "experience": input.weight_experience() / total,
                    "skill_density": input.weight_skill() / total,
                    "tenure": input.weight_tenure() / total,
                    "certification": input.weight_cert() / total,
                    "publication": input.weight_pub() / total,
                    "role_match": input.weight_role() / total
                }
                ahp_weights.set(new_weights)
        if input.reset_weights():
            ahp_weights.set(default_ahp_weights.copy())

    @output
    @render.data_frame
    def parser_table():
        df = dashboard_results()
        if df.empty:
            return pd.DataFrame()
        cols = [
            "file_name", "person_name", "current_role", "total_experience",
            "current_tenure", "certification_count", "publication_count", "skill_density"
        ]
        return df[cols]

    @reactive.Calc
    def fitment_score_table_data():
        df = dashboard_results()
        level = input.fitment_filter_score()
        return df if level == "All" else df[df["fitment_level"] == level]

    @output
    @render.data_frame
    def fitment_score_table():
        df = fitment_score_table_data()
        if not df.empty:
            return df[["person_name", "fitment_score", "fitment_level"]]
        else:
            return pd.DataFrame(columns=["person_name", "fitment_score", "fitment_level"])

app = App(app_ui, server)


Writing main_app.py


In [ ]:
import os
import pandas as pd
import tempfile
from difflib import SequenceMatcher

from shiny import App, reactive, render, ui
from shiny.types import FileInfo
from app_config import default_ahp_weights, col_map
from extractors import classify_fit

from ui_layout import app_ui
from extractors import (
    extract_text_from_docx, extract_text_from_pdf,
    extract_resume_features_combined as extract_resume_features,
    extract_jd_features, compute_ahp_fitment_score as compute_ahp_score,
    classify_fit   # Add this line
)
from db_utils import save_to_db


In [ ]:
#  Ensure Shiny app is saved (skip if already saved)
# with open("main_app.py", "w") as f:
#     f.write(shiny_code)

#  Install necessary packages
!pip install -q shiny pyresparser python-docx PyMuPDF ahpy

#  Kill any previous Shiny or Cloudflare tunnels
!pkill -f shiny || true
!pkill -f cloudflared || true

#  Download and set up Cloudflare tunnel binary
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/cloudflared

#  Clear old tunnel logs if any
!rm -f url.txt shiny_log.txt

#  Start Shiny app and cloudflared tunnel
import time
time.sleep(2)

!shiny run --host 0.0.0.0 --port 8084 main_app.py > shiny_log.txt 2>&1 &
!nohup cloudflared tunnel --url http://localhost:8084 --logfile url.txt > /dev/null 2>&1 &

#  Wait for tunnel to establish
time.sleep(8)

#  Display public URL
print("🔗 Public Shiny App Link:")
!grep -o 'https://[^ ]*\.trycloudflare\.com' url.txt | tail -n 1

#  Show last few log lines for debugging
print("\n📄 Last 10 lines of shiny_log.txt:")
!tail -n 10 shiny_log.txt


^C
^C
🔗 Public Shiny App Link:
https://officers-anxiety-bufing-locked.trycloudflare.com

📄 Last 10 lines of shiny_log.txt:


In [ ]:
from google.colab import files
files.download('/content/dashboard_shortlist_latest.csv')
